In [1]:
from model import *
from os.path import exists
import pandas as pd
import numpy as np

In [2]:
#generating GIFs
ps = [0.2, 0.5, 0.7]
rhos = [0.1, 0.2, 0.6]
MC_N = 100
average_velocity = mc_average_velocity_array(ps, rhos, MC_N, True)

Done: p = 0.2, rho = 0.1
Done: p = 0.2, rho = 0.2
Done: p = 0.2, rho = 0.6
Done: p = 0.5, rho = 0.1
Done: p = 0.5, rho = 0.2
Done: p = 0.5, rho = 0.6
Done: p = 0.7, rho = 0.1
Done: p = 0.7, rho = 0.2
Done: p = 0.7, rho = 0.6


In [3]:
#generating plot
ps = [0.2, 0.5, 0.7]
rhos = np.arange(0.05,1,0.05)
MC_N = 100
average_velocity = mc_average_velocity_array(ps, rhos, MC_N, False)

In [ ]:
plot_avg_velocity(average_velocity)